# Examples using HIF translators with HAT

[HAT (Hypergraph Analysis Toolbox)](https://hypergraph-analysis-toolbox.readthedocs.io/en/latest/) is a python package for studying the structure and dynamics of hypergraphs. This notebook provides examples to make `HAT.Hypergraph`s compatible with other libraries based on the HIF standard.

This notebook has three parts:
1. Install and import the packages
2. Load a HIF compatible JSON file
3. Build the `HAT.Hypergraph`
4. Demonstrate how to reproduce a HIF compatible hypergraph

## Load HIF File

In [1]:
import json
import os

import fastjsonschema
import hypernetx as hnx
from HAT import Hypergraph
from HAT.metrics import nonlinear_eigenvector_centrality

In [2]:
with open(os.path.join("data", "e-coli.json"), "r") as file:
    # Load the JSON data from the file
    hif_data = json.load(file)

## Convert to HAT

The `HAT.Hypergraph.from_hif` method converts the data loaded from a HIF `JSON` file into a Hypergraph.

In [3]:
HG = Hypergraph.from_hif(hif_data)

Metadata associated with each node can be viewed in the `nodes` and `edges` dataframes:

In [4]:
HG.nodes

,Nodes,name
0,3pg_c,3-Phospho-D-glycerate
1,succ_e,Succinate
2,etoh_e,Ethanol
3,13dpg_c,3-Phospho-D-glyceroyl phosphate
4,fdp_c,"D-Fructose 1,6-bisphosphate"
...,...,...
67,6pgl_c,"6-phospho-D-glucono-1,5-lactone"
68,acon_C_c,Cis-Aconitate
69,h_e,H+
70,o2_c,O2 O2


In [5]:
HG.edges

,edge,name,Nodes,Edges
0,SUCOAS_reverse,Succinyl-CoA synthetase (ADP-forming),"[19, 51, 32, 40, 36, 12]",0
1,EX_h2o_e,H2O exchange,[55],1
2,MALt2_2,Malate transport via proton symport (2 H),"[57, 69, 41, 43]",2
3,FUM,Fumarase,"[38, 17, 41]",3
4,H2Ot,H2O transport via diffusion,"[55, 38]",4
...,...,...,...,...
136,EX_mal__L_e,L-Malate exchange,[57],136
137,PGK_reverse,Phosphoglycerate kinase,"[19, 3, 0, 40]",137
138,PPC,Phosphoenolpyruvate carboxylase,"[38, 64, 11, 9, 51, 43]",138
139,PTAr_reverse,Phosphotransacetylase,"[36, 22, 23, 51]",139


## Convert Back to HIF

The `Hypergraph.to_hif` method converts the hypergraph object back to a python dictionary according to the HIF schema. This dictionary can be saved to a `JSON` file.

In [6]:
hif_output = HG.to_hif()

In [7]:
schema = json.load(open("../schemas/hif_schema_v0.1.0.json", "r"))
validator = fastjsonschema.compile(schema)

In [8]:
## The validator confirms the json read conforms to the HIF standard
output = validator(hif_output)

print("metadata: ", output["metadata"], "\n")
print("network-type: ", output["network-type"])

metadata:  {'uniform': False, 'order': -1, 'directed': False} 

network-type:  undirected


# Example Publications Dataset

In [ ]:
with open("data/publications.hif.json") as file:
    # Load the JSON data from the file
    hif_data_pub = json.load(file)
H_publications = Hypergraph.from_hif(hif_data_pub)

In [55]:
node_centrality, edge_centrality = nonlinear_eigenvector_centrality(H_publications, model='Linear')

In [82]:
print(f'The closeness centrality of the edges in hypergraph H is:')
print('-'*100)

sort_centr = sorted(enumerate(edge_centrality), key=lambda y: y[1], reverse=True)
for i, e in sort_centr:
    name = H_publications.edges["edge"][i]
    print(f'{name}: {e}')

The closeness centrality of the edges in hypergraph H is:
----------------------------------------------------------------------------------------------------
Hyper-YOLO: When Visual Object Detection Meets Hypergraph Computation: 0.10612699123500414
Beyond Graphs: Can Large Language Models Comprehend Hypergraphs?: 0.09402060213659152
LightHGNN: Distilling Hypergraph Neural Networks into MLPs for $100\times$ Faster Inference: 0.07687286653017818
LightHGNN: Distilling Hypergraph Neural Networks into MLPs for 100x Faster Inference: 0.07687286653017818
Hypergraph Dynamic System: 0.07424311714183425
Hypergraph Isomorphism Computation: 0.07424311714183425
Hypergraph-Based Multi-Modal Representation for Open-Set 3D Object Retrieval: 0.07251951901835571
Penalized Flow Hypergraph Local Clustering: 0.062398840422835486
Multi-View Time-Series Hypergraph Neural Network for Action Recognition: 0.05899934494091735
Hypergraph-based Multi-View Action Recognition using Event Cameras: 0.0587137833855035